In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'e:\\Projects\\text_summarizer_pipeline'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
 
        create_directories([self.config.artifacts_root]) 

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config["data_ingestion"]

        create_directories([config["root_dir"]])
        
        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

In [6]:
import os 
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(self.config.source_url, self.config.local_data_file)
            logger.info(f"Downloaded {filename} of size {get_size(Path(self.config.local_data_file))}")
        else:
            logger.info(f"{self.config.local_data_file} already exists")


    def unzip_file(self):
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)
        

In [12]:
try:
    config_mgr = ConfigurationManager()
    data_ingestion_config = config_mgr.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()

except Exception as e:
    raise e

[2023-06-13 11:22:11,782 - INFO - common - Yaml file: config\config.yaml loaded successfully]
[2023-06-13 11:22:11,784 - INFO - common - Yaml file: params.yaml loaded successfully]
[2023-06-13 11:22:11,785 - INFO - common - Created directory at: artifacts]
[2023-06-13 11:22:11,786 - INFO - common - Created directory at: artifacts/data_ingestion]
[2023-06-13 11:22:52,066 - INFO - 1817024924 - Downloaded artifacts/data_ingestion/data.zip of size 7718 kB]
